<a href="https://colab.research.google.com/github/AndyM2013/amazon_scraping/blob/main/amazon_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This side project is aiming to help retailers to scrape product data from Amazon by using Rainforest API https://www.rainforestapi.com/ Rainforest API provides free account registration with 100 API call credits. 

Imagine you have a product sheet from a distributor or a manufacturer and you would like to use the UPC numbers to search how the products sell on Amazon. The data you need including asin number, rating, totals rating, customer reviews, and best sellers rank.

The following code represents how to use Rainforest API to retrieve product data from Amazon. 

In [ ]:
import pandas as pd
import numpy as np
import requests
import json

In [ ]:
# construct file path
# file_name_list = ['file_1_name', 'file_2_name', 'file_3_name']
# prefix = '.xlsx' or '.csv'
# the constructed file path is under Google Colab default location
# for example, /content/file_1_name.csv
# modify the file path based on your own need like 
# change it to your Google Drive
# for example, /content/drive/MyDrive/file_1_name.csv

def filePathBuilder(file_name_list, prefix):
  file_path_list = []
  for file_name in file_name_list:
    file_path = "/content/" + file_name + prefix
    file_path_list.append(file_path)
  return file_path_list

In [ ]:
# In my sample data sheet, the UPC number column was named 
# "UPC #" or "UPC", so I updated it to "upc"

# the product sheet might contain a varitey of columns, but 
# we only need upc in our case

# Rainforest API also support search by URL, ASIN, GTIN, EAN

def prepareDataframe(file_path):
    df = pd.read_excel(file_path)
    if "UPC #" in df.columns:
      df = df[['UPC #']]
    elif "UPC" in df.columns:
      df = df[['UPC']]
    df.columns = ['upc']
    # drop rows contain empty upc value
    df.dropna(inplace=True)
    indexNames = df[ (df['upc'] == 'nan') & (df['upc'] == 0.0) ].index
    df.drop(indexNames , inplace=True)
    df['upc'] = df['upc'].astype(int) # convert upc column to integer
    return df

In [ ]:
# prepare file path list for our source files

file_name_list = ["file_1_name",
                  "file_2_name",
                  "file_3_name"]
file_path_list = filePathBuilder(file_name_list, ".xlsx")

In [ ]:
# prepare file path list which will be used to save our data

final_file_path_list = filePathBuilder(file_name_list, ".csv")

In [ ]:
# loop the file path list and process each file

for i in range(len(file_path_list)):
  print(f"<-------Start scraping {file_path_list[i]}--------->")
  df = prepareDataframe(file_path_list[i]) # dataframe preparation 
  final_data = {} # to store all the scraped data as a dictionary

  # iterate dataframe row by row
  for row in df.itertuples(index = True, name ='Pandas'):
    upc = str(row.upc) # convert upc from integer to string
    params = {
      # this api_key doesn't work, replace it with YOUR OWN KEY
      'api_key': '8FFB0ACB661848A5B9A8ATRCE1C827AA1B2', 
      'type': 'product',
      # change domain if you want search market in other region
      # for instance, amazon.com, amazon.co.uk
      'amazon_domain': 'amazon.ca', 
      'gtin': upc
    }

    # call the API and get respone result
    # result data type is json
    api_result = requests.get('https://api.rainforestapi.com/request', params)
    data = api_result.json()

    # find the data we want from the respone result
    print(f"Get API respone for UPC number -----> {upc}")

    # response result with success is True means the API find the product 
    # from the selected Amazon domain 
    if data.get("request_info").get("success"):
      print("api success")

      # access the asin, rating and ratings_total
      asin = data.get("product").get("asin")
      rating = data.get("product").get("rating")
      ratings_total = data.get("product").get("ratings_total")
      print(f"asin: {asin}; upc: {upc}; rating: {rating}; ratings_total: {ratings_total}")

      # check if the response result includes the specifications section 
      # which might contains Customer reviews and Best sellers rank data

      # if cannot find the customer reviews and best sellers rank data, then
      # assign None to the two variables
      if data.get("product").get("specifications") != None:
        print("find specification section")
        for i in data.get("product").get("specifications"):
          if i.get("name") == "Customer Reviews":
            print(i["value"])
            customer_reviews = i["value"]
          elif i.get("name") == "Best Sellers Rank":
            print(i['value'])
            best_sellers_rank = i['value']
          else:
            customer_reviews = None
            best_sellers_rank = None
            print("not find Customer reviews and best sellers rank in specifications")

      # best sellers rank might also exist under the bestsellers_rank section
      # check if the bestsellers_rank section exist, then looking for 
      # the product category and rank. 
      if data.get("product").get("bestsellers_rank") != None:
        print("find best sellers rank section")
        ranks = data.get("product").get("bestsellers_rank")
        rank_category = ranks[0].get("category")
        rank = ranks[0].get("rank")
        print(f"rank category: {rank_category}; rank: {rank}")
      else:
        rank_category = None
        rank = None
        print("not find bestsellers_rank")

      # save our data to a dictionary with key is the upc number 
      # and all the values are saved to a list
      final_data[upc] = [asin, upc, rating, ratings_total, customer_reviews, best_sellers_rank, rank_category, rank]
    else:
      print(f"api failed at upc {upc}")
    
  # convert our organized data to a dataframe and save it to a csv file
  results_df = pd.DataFrame.from_dict(final_data, orient='index',
                                      columns=['asin', 'upc', 'rating', 'ratings_total', 
                                              'Customer reviews', 'best sellers rank',
                                              'bestsellers_rank_category', 'bestsellers_ranks'])

  results_df.to_csv(final_file_path_list[i], index=False)
  print("<-----------csv file saved!---------------->")
print("<-----------All done!---------------->")